In [16]:
import numpy as np 
import pandas as pd

In [47]:

## get cursor to start querying the server
database = "BikeStores"
table = "sales.orders"
user = "admin"
password  = "demarchi481"
server = "bikestores.cv0paa5jyvim.us-east-2.rds.amazonaws.com"

conn=pyodbc.connect('DRIVER={SQL SERVER};SERVER='+server+';DATABASE='+database+';uid='+user+';pwd='+ password) 

cursor = conn.cursor()
cursor.execute('SELECT * FROM sales.orders')
data = cursor.fetchall()
df = pd.DataFrame(data)
columns = [desc[0] for desc in cursor.description]

dframe = pd.DataFrame.from_records(data,
                               columns = [desc[0] for desc in cursor.description])
dframe.shape

#for row in cursor:
#    print(row)


(1615, 8)

In [53]:
#Connection to MSSQL database 
def connect_db():
    database = "BikeStores"
    user = "admin"
    password  = "demarchi481"
    server = "bikestores.cv0paa5jyvim.us-east-2.rds.amazonaws.com"
    conn=pyodbc.connect('DRIVER={SQL SERVER};SERVER='+server+';DATABASE='+database+';uid='+user+';pwd='+ password)
    return conn

In [54]:
#Convert cursor SQL to Data Frame
def to_frame(sql):
    conn = connect_db()
    cursor.execute(sql)
    data = cursor.fetchall()
    if data:
        df = pd.DataFrame.from_records(data,
                               columns = [desc[0] for desc in cursor.description])
    else:
        df = pd.DataFrame()
    return df

In [67]:
sql = 'SELECT table_name FROM information_schema.tables'
df = to_frame(sql)
df

,table_name
0,categories
1,brands
2,products
3,customers
4,stores
5,staffs
6,orders
7,order_items
8,stocks


In [63]:
sql = 'SELECT * FROM sales.orders'
df = to_frame(sql)
df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6
...,...,...,...,...,...,...,...,...
1610,1611,6,3,2018-09-06,2018-09-06,None,2,7
1611,1612,3,3,2018-10-21,2018-10-21,None,1,3
1612,1613,1,3,2018-11-18,2018-11-18,None,2,6
1613,1614,135,3,2018-11-28,2018-11-28,None,3,8


In [68]:
## Explore to create Data Mart: Orders
sql = """SELECT * 
         FROM sales.orders o
         join sales.order_items oi on o.order_id = oi.order_id
         join production.products p on oi.product_id = p.product_id
         join sales.customers c on o.customer_id = c.customer_id
         join sales.staffs s on o.staff_id = s.staff_id
         join sales.stores st on st.store_id = o.store_id"""
df = to_frame(sql)
df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,order_id,item_id,...,store_id,manager_id,store_id,store_name,phone,email,street,city,state,zip_code
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,1,1,...,1,1,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
1,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,1,2,...,1,1,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
2,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,1,3,...,1,1,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
3,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,1,4,...,1,1,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
4,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,1,5,...,1,1,1,Santa Cruz Bikes,(831) 476-4321,santacruz@bikes.shop,3700 Portola Drive,Santa Cruz,CA,95060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4717,1614,135,3,2018-11-28,2018-11-28,None,3,8,1614,2,...,3,1,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088
4718,1614,135,3,2018-11-28,2018-11-28,None,3,8,1614,3,...,3,1,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088
4719,1615,136,3,2018-12-28,2018-12-28,None,3,8,1615,1,...,3,1,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088
4720,1615,136,3,2018-12-28,2018-12-28,None,3,8,1615,2,...,3,1,3,Rowlett Bikes,(972) 530-5555,rowlett@bikes.shop,8000 Fairway Avenue,Rowlett,TX,75088
